In [1]:
import os
import pandas as pd
import numpy as np
import joblib
 
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model_file = "heart_model.pkl"
pipeline_file = "heart_pipeline.pkl"

def build_pipeline(num_attri):
    num_pipeline = Pipeline([
        ("imputer",SimpleImputer(strategy="median")),
        ("scaling",StandardScaler())
    ])

    my_pipeline = ColumnTransformer([
        ("num",num_pipeline,num_attri),
    ])
    return my_pipeline

if not os.path.exists(model_file):
    df = pd.read_csv(r"D:\sci ml\archive.zip")
    split = StratifiedShuffleSplit(n_splits =1,test_size =0.2,random_state = 42)
    for train,test in split.split(df,df["target"]):
        df.iloc[test].drop("target", axis=1).to_csv("heart_input_data.csv", index=False)  # Save features only
        train_strat = df.iloc[train]

    
    df = train_strat.copy()
    labels = df["target"].copy()
    features = df.drop(["target"],axis =1)
    
    num_attri = features.columns.tolist()
    pipeline = build_pipeline(num_attri)
    prepared = pipeline.fit_transform(features)
    model = RandomForestClassifier()
    model.fit(prepared,labels)
    
    joblib.dump(model,model_file)
    joblib.dump(pipeline,pipeline_file)
    print("Model trained & saved")

else:
    model =  joblib.load(model_file)
    pipeline = joblib.load(pipeline_file)

    input_data = pd.read_csv("heart_input_data.csv")
    processed_input_data = pipeline.transform(input_data)
    predictions = model.predict(processed_input_data)
    input_data["target"]  = predictions

    input_data.to_csv("heart_disease_prediction.csv",index = False)
    print("Inference Completed ")

Inference Completed 
